In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
import numpy as np
# Import various methods.
import beta
import bumblepuppy
import dllu
import greenberg
import iou
import memory
import randomforest
import transition
import iocaine
import time

In [16]:
# Write two classes, observation and configuration, in order for the script to run.
class observe():
    def __init__(self):
        self.lastOpponentAction = 0
        self.step = 1    
class config():
    def __init__(self):
        self.lastOpponentAction = 0
        self.step = 1  

In [24]:
# Use Proportional Representation to find most favorable move.
def proportionalRepresentation(array_of_moves, array_of_won_rounds, step):
    def filterWinrates(moves, winrates):
        indexes = []
        for i in range(len(winrates)):
            # If one move's winrate is below 25%, remove it from the calculation.
            if winrates[i] < 0.25:
                indexes.append(i)
        moves = np.delete(moves, indexes)
        winrates = np.delete(winrates, indexes)
        return moves, winrates

    array_of_win_rates = array_of_won_rounds / step
    # Filter out the methods that has very low win rate.
    array_of_moves, array_of_win_rates = filterWinrates(array_of_moves, array_of_win_rates)

    # If no method has threshold winrate, return a random move.
    if len(array_of_win_rates) == 0:
        return np.random.randint(3)
    move_dict = {0: np.array([]), 1: np.array([]), 2:np.array([])}
    for index, element in enumerate(array_of_moves):
        move_dict[element] = np.append(move_dict[element], array_of_win_rates[index])
    for move in move_dict:
        if len(move_dict[move]) == 0:
            move_dict[move] = 0
        else:
            move_dict[move] = sum(move_dict[move]) / len(move_dict[move])
    best_move = 0
    best_rating = -1
    for move in move_dict:
        if move_dict[move] > best_rating:
            best_move = move
            best_rating = move_dict[move]
    return best_move

def updateWinRate(array_of_moves, opponent_move, array_won_rounds):
    current_round = np.array([0] * len(array_of_moves))
    for index, move in enumerate(array_of_moves):
        if move == (opponent_move + 1) % 3:
            current_round[index] = 1
    return array_won_rounds + current_round

main_step = 0
main_won_rounds = np.array([0])
main_method_steps = np.array([0])

def main(observation, configuration):
    global main_step
    global main_won_rounds
    global main_method_steps
    # If not first step, update # of won round first.
    if main_step > 0:
        main_won_rounds = updateWinRate(main_method_steps, 
                                       int(observation.lastOpponentAction), 
                                       main_won_rounds)

    # Compute moves generated by each method.
    start = time.time()
    step_random_forest = randomforest.random_forest_random(observation, configuration)
    end = time.time()
    print(end - start)
    '''
    step_transition = transition.transition_agent(observation, configuration)
    step_multi_armed = beta.multi_armed_bandit_agent (observation, configuration)
    step_dllu1 = dllu.run(observation, configuration)
    step_greenberg = greenberg.greenberg_agent(observation, configuration)
    step_IOU = iou.agent(observation, configuration)
    step_memory = memory.my_agent(observation, configuration)
    step_bumblepuppy = bumblepuppy.run(observation, configuration)
    step_locaine = iocaine.iocaine_agent(observation, configuration)
    '''
    # Have all the moves in a np array.
    main_method_steps = np.array([step_random_forest])

    # If not first step, find current optimized step.
    if main_step > 0:
        selected_step = proportionalRepresentation(main_method_steps, main_won_rounds, main_step)
        main_step += 1
        return selected_step
    # Otherwise, return random number.
    else:
        main_step += 1
        return np.random.randint(3)

In [30]:
# Test script.

obs = observe()
config = 0

main(obs, config)

0.0


0